# PHASE 0: Problem Framing & Churn Definition

## Big-Tech-Grade User Retention & Churn Prediction System

---

**Author**: Senior Data Scientist  
**Date**: February 2026  
**Objective**: Define churn in business terms, establish the problem framework, and set success criteria

---

## Table of Contents
1. [Business Context](#1-business-context)
2. [Churn Definition](#2-churn-definition)
3. [Business Cost Framework](#3-business-cost-framework)
4. [Success Metrics](#4-success-metrics)
5. [Assumptions & Limitations](#5-assumptions--limitations)

---

## 1. Business Context

### 1.1 The Retention Problem

In e-commerce, **customer retention is 5-25x cheaper than acquisition**. Yet most companies focus disproportionately on acquisition.

| Metric | Industry Average | Top Performers |
|--------|-----------------|----------------|
| Annual Churn Rate | 20-30% | 5-10% |
| Customer Acquisition Cost (CAC) | $50-150 | $30-80 |
| Customer Lifetime Value (CLV) | $150-400 | $500+ |
| Retention ROI | 3-5x | 8-12x |

### 1.2 Why Predict Churn?

**Reactive Approach** (Traditional):
- Customer stops buying  Company notices  Too late to act
- Win-back campaigns have <5% success rate

**Proactive Approach** (This Project):
- Predict churn risk  Intervene early  Retain customer
- Early intervention has 20-40% success rate

### 1.3 Business Questions We Must Answer

1. **Who** is likely to churn? (Prediction)
2. **When** will they churn? (Time-to-event)
3. **Why** are they churning? (Root cause)
4. **What** can we do about it? (Intervention)
5. **How much** should we invest? (ROI optimization)

---

## 2. Churn Definition

### 2.1 Formal Definition

```
CHURN = A customer who has NOT made any purchase in 60 consecutive days
```

### 2.2 Why 60 Days?

The choice of churn window is **critical** and must be data-driven:

| Window | Pros | Cons | Use Case |
|--------|------|------|----------|
| **30 days** | Quick detection | High false positives (seasonal buyers) | Subscription services |
| **60 days** | Balanced signal | Moderate delay | **General e-commerce** |
| **90 days** | Few false positives | Late detection, harder to recover | Luxury/high-ticket items |

**Our Rationale for 60 Days:**

1. **Purchase Cycle Analysis**: Typical retail customers purchase every 30-45 days. A 60-day gap represents ~1.5-2x normal cycle.

2. **Actionability**: 60 days provides enough lead time for:
   - Email re-engagement campaigns (2-3 weeks)
   - Personalized offers (1-2 weeks)
   - Customer success outreach (1 week)

3. **Statistical Significance**: Based on industry benchmarks, probability of return drops significantly after 60 days:
   - Day 30: 65% return probability
   - Day 60: 35% return probability
   - Day 90: 15% return probability

4. **Business Alignment**: Most retail retention budgets operate on quarterly cycles; 60-day detection allows for one intervention cycle before quarter-end reviews.

### 2.3 Operational Definition

```python
# Pseudo-code for churn labeling
def is_churned(customer_id, observation_date, lookforward_window=60):
    """
    Returns True if customer makes NO purchase in the next 60 days
    after observation_date.
    """
    future_purchases = get_purchases(
        customer_id,
        start=observation_date,
        end=observation_date + 60_days
    )
    return len(future_purchases) == 0
```

---

## 3. Business Cost Framework

### 3.1 Cost Matrix

This is the **most important business decision** in the project.

| Prediction | Reality | Outcome | Business Cost |
|------------|---------|---------|---------------|
| **Not Churn** | **Churns** | False Negative (FN) | **HIGH**: Lost CLV (~$200) |
| **Churn** | **Doesn't Churn** | False Positive (FP) | **LOW**: Wasted incentive (~$20) |
| **Not Churn** | **Doesn't Churn** | True Negative (TN) | None (status quo) |
| **Churn** | **Churns** | True Positive (TP) | **BENEFIT**: Saved revenue (~$80*) |

*Assuming 40% retention success rate Ã $200 CLV = $80 expected value saved

### 3.2 Cost Assumptions

| Parameter | Value | Source/Justification |
|-----------|-------|---------------------|
| Average CLV | $200 | Based on Online Retail II avg customer value |
| Retention Incentive Cost | $20 | Typical: 10% discount + email cost |
| Retention Success Rate | 40% | Industry benchmark for early intervention |
| False Positive Tolerance | 5:1 | We can afford 5 unnecessary offers per 1 saved customer |

### 3.3 Cost-Benefit Analysis

**Key Insight**: The cost ratio determines our threshold strategy.

```
Cost Ratio = Cost(FN) / Cost(FP) = $200 / $20 = 10:1
```

This means:
- We should **favor recall over precision**
- Missing a churner (FN) costs 10x more than a false alarm (FP)
- Optimal threshold will be **below 0.5** (recall-biased)

### 3.4 Expected Business Impact

**Scenario**: 10,000 customers, 25% churn rate (2,500 churners)

| Approach | Churners Caught | False Alarms | Net Benefit |
|----------|-----------------|--------------|-------------|
| No Model | 0 | 0 | -$500,000 (all churn) |
| Random (50%) | 1,250 | 3,750 | -$175,000 |
| Our Model (80% recall) | 2,000 | 1,500 | **+$130,000** |

**ROI Calculation**:
- Customers saved: 2,000 Ã 40% = 800
- Revenue retained: 800 Ã $200 = $160,000
- Intervention cost: 3,500 Ã $20 = $70,000 (TP + FP)
- **Net Benefit: $90,000**

---

## 4. Success Metrics

### 4.1 Technical Metrics

| Metric | Target | Why It Matters |
|--------|--------|----------------|
| **Recall** |  75% | Catch most churners |
| **Precision** |  50% | Limit wasted interventions |
| **ROC-AUC** |  0.80 | Model discriminative ability |
| **PR-AUC** |  0.60 | Performance on imbalanced data |

### 4.2 Business Metrics

| Metric | Target | Calculation |
|--------|--------|-------------|
| **Retention Rate Lift** | +15% | (New retention - Baseline) / Baseline |
| **Cost per Retained Customer** | < $50 | Total intervention cost / Customers saved |
| **Intervention ROI** | > 3x | Revenue saved / Intervention cost |

### 4.3 Why Accuracy is NOT Sufficient

**Example**: With 25% churn rate:
- A model predicting "no churn" for everyone achieves **75% accuracy**
- But it has **0% recall** (catches no churners)
- Business impact: **Zero** (worse than useless)

**We optimize for business value, not accuracy.**

---

## 5. Assumptions & Limitations

### 5.1 Explicit Assumptions

| Assumption | Impact | Mitigation |
|------------|--------|------------|
| Churn is predictable from behavior | Core assumption | Validate with feature importance |
| Past behavior predicts future | Standard ML assumption | Use time-based validation |
| CLV is uniform (~$200) | May over/under-value segments | Build segment-specific CLV |
| Retention rate is 40% | Critical for ROI | A/B test to validate |
| 60-day window is appropriate | Defines the problem | Sensitivity analysis |

### 5.2 Known Limitations

1. **Data Limitations**:
   - Single channel (no mobile app, physical store)
   - No customer demographics
   - No external factors (competition, economy)
   - UK-centric (cultural purchase patterns)

2. **Model Limitations**:
   - Point-in-time prediction (not dynamic)
   - Binary classification (not survival analysis)
   - No causal inference (correlation only)

3. **Business Limitations**:
   - Assumes intervention capacity
   - Assumes customer contact info available
   - No competitive intelligence

### 5.3 Out of Scope

- Real-time scoring
- A/B testing framework
- Intervention recommendation engine
- Customer lifetime value modeling
- Survival/time-to-event analysis

---

## 6. Summary: Problem Statement

### Final Problem Statement

> **Build a machine learning system that predicts which customers will churn (no purchase in 60 days) with at least 75% recall and 50% precision, enabling proactive retention interventions that generate positive ROI.**

### Key Deliverables

1.  Churn prediction model (XGBoost)
2.  Feature importance analysis
3.  Cost-optimized decision threshold
4.  Customer risk segmentation
5.  Retention strategy recommendations

### Next Steps

 **Phase 1**: Data Understanding & Validation

---

## Phase 0 Checklist

- [x] Defined churn (60-day inactivity)
- [x] Justified churn window choice
- [x] Established cost matrix
- [x] Set success metrics (technical + business)
- [x] Documented assumptions
- [x] Acknowledged limitations
- [x] Created formal problem statement

**Phase 0 Status: COMPLETE** 